# <p style="text-align: center;, font-style: strong;">Partie 2 : MNIST with Multiple Layer Perceptron (MLP)</p>

### <p style="text-align: center;">(Almond 0.9.1, Scala 2.12.10)</p>


In this notebook, we will work on a dataset popularly known as MNIST. It is a dataset consisting of handwritten digits. More details can be found here: http://yann.lecun.com/exdb/mnist/


We will train a model to recognize the handwritten digits into correct digit:


<img src="../resources/mnist_mlp.png" alt="MNIST Dataset and Number Classification" style="width: 800px;"/>  

Ref: https://towardsdatascience.com/image-classification-in-10-minutes-with-mnist-dataset-54c35b77a38d

## Dependencies

Usual suspects in Scala TF setup

In [ ]:
interp.load.ivy(coursierapi.Dependency.of("org.platanios", "tensorflow_2.12", "0.4.1").withClassifier("linux-cpu-x86_64"))
interp.load.ivy("org.platanios" %% "tensorflow-data" % "0.4.1")

In [ ]:
import java.nio.file.Paths

import org.platanios.tensorflow.api._

import org.platanios.tensorflow.api.tf
import org.platanios.tensorflow.api.tensors.Tensor
import org.platanios.tensorflow.api.core.Shape
import org.platanios.tensorflow.api.core.Indexer._
import org.platanios.tensorflow.api.core.client.Session
import org.platanios.tensorflow.data.image.MNISTLoader

import org.platanios.tensorflow.api.learn.layers.{ Flatten, Input, Linear, ReLU, SparseSoftmaxCrossEntropy, Mean }
import org.platanios.tensorflow.api.learn.{ Model, StopCriteria }
import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator


## Display MNIST Dataset

Define and execute a function to display the first 20 images found in the written digits images database.  
MNIST dataset is already downloaded in resource directory from http://yann.lecun.com/exdb/mnist/  
We will use `org.platanios.tensorflow.data.image.MNISTLoader` class to load the dataset.  

Original image is of `28 x 28` dimension with `grayscale` channel. And use `tf.image.encodePng` to transform it into PNG image.  
Displayed image is resized into size of `100 x 100`.


In [ ]:
{{
def displayNumberMNIST(nb: Int) {
    val dataset = MNISTLoader.load(Paths.get("../resources/dataset"))
    val images = dataset.trainImages
    val imagesToDisplay = images.slice(0 :: nb, ::, ::)
    for (index <- 0 until nb) {
        val png = Session().run(fetches = tf.decodeRaw[Byte](tf.image.encodePng(imagesToDisplay(index).reshape(Shape(28, 28, 1)))))
        Image(png.entriesIterator.toArray).withFormat(Image.PNG).withWidth(100).withHeight(100).display 
    }
}
displayNumberMNIST(20)
}}

## Dataset details
1. __train-images-idx3-ubyte.gz__: 60k images of dim 28x28 for training the network
2. __train-labels-idx1-ubyte.gz__: 60k labels (digit between 0 and 9: output of network) for above training images
3. __t10k-images-idx3-ubyte.gz__: 10k images of dim. 28x28 for testing purpose
4. __t10k-labels-idx1-ubyte.gz__: 10k labels for test dataset


In [ ]:
val dataset = MNISTLoader.load(Paths.get("../resources/dataset"))

### Data preparation for training

We need to combine training image and its label to create a  proper training dataset for the network.  
We do some preprocessing e.g. shuffling, iteraor looping, batching for  training.  
We create batches of 256 images at a time, so that each training iteration will use 256 images.  
We are using tensorflow `tf.data.Dataset` class API to do the above data processing:  

- `zip`: creates a new dataset by zipping training image and its label  
- `repeat`: repeast the dataset whenever the network ask new data for training and the network has already seen all the dataset  
- `shuffle`: it will shuffle the dataset  
- `batch`: combines consecutive elements of the dataset into one element, so that in each iteration when the network ask for data, it will get # of data defined in a batch  
- `prefetch`: prefetches elements from the dataset  


For more details follow below references:  
  - https://www.tensorflow.org/api_docs/python/tf/data/Dataset
  - https://www.tensorflow.org/guide/data
  - https://adventuresinmachinelearning.com/tensorflow-dataset-tutorial/
  
  

In [ ]:
val dataset = MNISTLoader.load(Paths.get("../resources/dataset"))
val trainImages = tf.data.datasetFromTensorSlices(dataset.trainImages.toFloat)

val trainLabels = tf.data.datasetFromTensorSlices(dataset.trainLabels.toLong)
val trainData =
  trainImages.zip(trainLabels)
      .repeat()
      .shuffle(60000)
      .batch(256)
      .prefetch(10)

### Model definition

We define here the shape for input data and the Neural Network topology.

`Flatten[Float]("Input/Flatten")` layer: We start by reshaping the `28x28` matrix as a flat vector of size 784.

`Linear[Float]("Layer_0", 256)` layer: Then we connect these cells to a single 256 nodes layer, fully connected.

`Linear[Float]("OutputLayer", 10)` layer: Then again connect these to a 10-cells output (because we have 10 classes = 10 digits)

As shown in the diagram below:

<img src="../resources/mnist_2layers_new.png" alt="MNIST Dataset and Number Classification" style="width: 600px;"/>  

Ref: https://ml4a.github.io/ml4a/looking_inside_neural_nets/



### To Do: Excercise

- Add a second fully connected Layer
- test the use of a Rectifying Linear Unit at each layer output like `ReLU[Float]("Layer_0/Activation")`
- test more steps...

In [ ]:
// Create the MLP model.
val input = Input(FLOAT32, Shape(-1, 28, 28))
val trainInput = Input(INT64, Shape(-1))
val layer = Flatten[Float]("Input/Flatten") >> 
    Linear[Float]("Layer_0", 256)  >>
    Linear[Float]("OutputLayer", 10)

### Loss, optimization and wrapping in an Estimator

- __Loss__: For loss calculation, we are using softmax with cross entropy combined layer together with a mean layer.  
Measures the probability error in discrete classification tasks in which the classes are mutually exclusive.  
Details: https://www.tensorflow.org/api_docs/python/tf/compat/v1/losses/sparse_softmax_cross_entropy


- __Optimizer__: Optimizer that implements the Adam algorithm.
Details: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam

- __Model__: Define a supervised model with its required parameters

- __InMemoryEstimator__: In-memory estimator which is used to train, use, and evaluate TensorFlow models, and uses an underlying TensorFlow session that it keeps alive throughout its lifetime.  
Details: [InMemoryEstimator.scala](https://github.com/eaplatanios/tensorflow_scala/blob/master/modules/api/src/main/scala/org/platanios/tensorflow/api/learn/estimators/InMemoryEstimator.scala)


In [ ]:
val loss = SparseSoftmaxCrossEntropy[Float, Long, Float]("Loss") >> Mean("Loss/Mean")
val optimizer = tf.train.Adam()
val model = Model.simpleSupervised(input, trainInput, layer, loss, optimizer)

// Create an estimator and train the model.
val estimator = InMemoryEstimator(model)

### Training!
Train the model and stop it after some no. of iterations.

In [ ]:
estimator.train(() => trainData, StopCriteria(maxSteps = Some(2500)))

### Metrics for model quality: accuracy
Evaluate the model's accuracy on test dataset

In [ ]:
def accuracy(images: Tensor[UByte], labels: Tensor[UByte]): Float = {
    val predictions = estimator.infer(() => images.toFloat)
    predictions
      .argmax(1).toUByte
      .equal(labels).toFloat
      .mean().scalar
}

println(s"Train accuracy = ${accuracy(dataset.trainImages, dataset.trainLabels)}")
println(s"Test accuracy = ${accuracy(dataset.testImages, dataset.testLabels)}")

## Test results
Check the performance of trained model on test dataset

In [ ]:
val images = dataset.testImages

def inferOnSelectedImage(indexes: Seq[Int], images: Tensor[UByte]) {
    indexes.foreach { index => 
        val imageToInfer = images.slice(index, ::, ::).reshape(Shape(1, 28, 28))
        val predictions = estimator.infer(() => imageToInfer.toFloat)
        println(s"Label infered: ${predictions.argmax(1).scalar}")
        val png = Session().run(fetches = tf.decodeRaw[Byte](tf.image.encodePng(imageToInfer.reshape(Shape(28, 28, 1)))))
        Image(png.entriesIterator.toArray).withFormat(Image.PNG).withWidth(100).withHeight(100).display 
    }
}

inferOnSelectedImage((30 to 40), images)
